In [1]:
from functools import cache
from typing import *
from math import inf
from collections import Counter, defaultdict
from bisect import bisect_left, bisect_right
from sortedcontainers import SortedList, SortedDict, SortedSet
from itertools import accumulate

# 数据结构

In [ ]:
# 树状数组 离散化
# 树状数组模板（维护前缀最大值）
class BIT:
    def __init__(self, n: int):
        self.tree = [-inf] * n

    def update(self, i: int, val: int) -> None:
        while i < len(self.tree):
            self.tree[i] = max(self.tree[i], val)
            i += i & -i

    def pre_max(self, i: int) -> int:
        mx = -inf
        while i > 0:
            mx = max(mx, self.tree[i])
            i &= i - 1
        return mx



In [ ]:
# 树状数组
## 通常是边更新，边查询的情况  随时更新区间范围
class BIT:
    def __init__(self, n: int):
        self.tree = [0] * n  # 树状数组
        self.original = [0] * n  # 原数组

    def update(self, i: int, val: int) -> None:
        self.original[i] = max(self.original[i], val)
        while i < len(self.tree):
            self.tree[i] = max(self.tree[i], val)
            i += i & -i

    def query_max(self, L: int, R: int) -> int:
        mx = 0
        while R >= L:
            r = R & (R - 1)
            # 查询先进行比较，看下一个r在不在查询范围内
            if r >= L:
                # 在查询范围内，直接从树状数组拿值比较
                mx = max(mx, self.tree[R])
                R = r
            else:
                # 只走一步，从原数组拿值比较
                mx = max(mx, self.original[R])
                R -= 1
        return mx

    # 统计 <= R 的元素个数
    def query(self, R: int) -> int:
        res = 0
        while R > 0:
            res += self.tree[R]
            R &= (R - 1)
        return res

    def add(self, i: int, val: int) -> None:
        while i < len(self.tree):
            self.tree[i] += val
            i += i & -i


In [ ]:
# 二维树状数组 矩阵更新单个值，求区间面积
class BIT2D:
    def __init__(self, n, m):
        self.n = n
        self.m = m
        self.BIT = [[0] * (m + 1) for _ in range(n + 1)]  # 初始化树状数组

    def update(self, x, y, delta):
        """在 (x, y) 位置增加 delta"""
        i = x
        while i <= self.n:
            j = y
            while j <= self.m:
                self.BIT[i][j] += delta
                j += j & -j  # 更新 y 坐标
            i += i & -i  # 更新 x 坐标

    def query(self, x, y):
        """查询 (1, 1) 到 (x, y) 的区域和"""
        sum = 0
        i = x
        while i > 0:
            j = y
            while j > 0:
                sum += self.BIT[i][j]
                j -= j & -j  # 退回 y 坐标
            i -= i & -i  # 退回 x 坐标
        return sum

    def range_query(self, x1, y1, x2, y2):
        """查询 (x1, y1) 到 (x2, y2) 的区域和"""
        return (self.query(x2, y2)
                - self.query(x1 - 1, y2)
                - self.query(x2, y1 - 1)
                + self.query(x1 - 1, y1 - 1))

# 示例用法
n, m = 5, 5
bit = BIT2D(n, m)

# 更新点 (2, 3) 的值加 5
bit.update(2, 3, 5)

# 查询区域 (1, 1) 到 (3, 3) 的和
result = bit.range_query(1, 1, 3, 3)
print(result)  # 输出某个区域的和

In [ ]:
# 并查集
## 非数组情况的并查集
class UnionFind:
    def __init__(self):
        self.parent = {}
        self.rank = {}

    def find(self, x):
        if x not in self.parent:
            self.parent[x] = x
            self.rank[x] = 0
        elif self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]

    def union(self, x, y):
        x_root = self.find(x)
        y_root = self.find(y)

        if x_root == y_root:
            return

        if self.rank[x_root] < self.rank[y_root]:
            self.parent[x_root] = y_root
        elif self.rank[x_root] > self.rank[y_root]:
            self.parent[y_root] = x_root
        else:
            self.parent[y_root] = x_root
            self.rank[x_root] += 1

## 数组版
class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
        self.size = [1] * n

    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])  # 路径压缩
        return self.parent[x]

    def union(self, x, y):
        rootX = self.find(x)
        rootY = self.find(y)
        if rootX != rootY:
            if self.size[rootX] < self.size[rootY]:
                rootX, rootY = rootY, rootX
            self.parent[rootY] = rootX
            self.size[rootX] += self.size[rootY]

    def getSize(self, x):
        root = self.find(x)
        return self.size[root]
# https://leetcode.cn/problems/making-a-large-island/solutions/2808887/jian-ji-gao-xiao-ji-suan-dao-yu-de-mian-ab4h7/

In [1]:
# 小技巧


IndentationError: unexpected indent (3200444650.py, line 2)

# 位运算
```
交换律和结合律
a & b = b & a
( a & b ) & c = a & ( b & c )

分配律
a & ( b | c ) = ( a & b ) | ( a & c )
a ^ ( b | c ) = ( a ^ b) | (a ^ c)

其它性质
a | ( a & b ) = a
a & ( a | b ) = a
```


In [ ]:
# 前缀和

# 二维前缀和
class PrefixSum2D:
    def __init__(self, matrix):
        if not matrix or not matrix[0]:
            self.prefix_sum = []
            return

        self.n = len(matrix)
        self.m = len(matrix[0])
        self.prefix_sum = [[0] * (self.m + 1) for _ in range(self.n + 1)]

        # 构建前缀和矩阵
        for i in range(1, self.n + 1):
            for j in range(1, self.m + 1):
                self.prefix_sum[i][j] = (matrix[i - 1][j - 1]
                                         + self.prefix_sum[i - 1][j]
                                         + self.prefix_sum[i][j - 1]
                                         - self.prefix_sum[i - 1][j - 1])

    def query(self, x1, y1, x2, y2):
        """查询从 (x1, y1) 到 (x2, y2) 的区域和"""
        return (self.prefix_sum[x2 + 1][y2 + 1]
                - self.prefix_sum[x1][y2 + 1]
                - self.prefix_sum[x2 + 1][y1]
                + self.prefix_sum[x1][y1])

# 示例用法
matrix = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
]

prefix_sum = PrefixSum2D(matrix)

# 查询区域 (1, 1) 到 (2, 2) 的和
result = prefix_sum.query(1, 1, 2, 2)
print(result)  # 输出: 28 (5 + 6 + 8 + 9)

In [ ]:
# 最近公共祖先

In [ ]:
# 数论
## math.ceil() 向上取整

# 筛选质数的方法 埃拉托斯特尼筛法
def sieve_of_eratosthenes(n):
    is_prime = [True] * (n + 1)
    is_prime[0] = is_prime[1] = False  # 0 和 1 不是质数

    for i in range(2, int(n**0.5) + 1):
        if is_prime[i]:
            for j in range(i * i, n + 1, i):
                is_prime[j] = False

    return [i for i in range(n + 1) if is_prime[i]]

# DP

In [ ]:
# 区间DP
# 思想就是随着操作的进行，判断的范围越来越小，所以小长度开始遍历，逐步化解更大范围的问题
# 模板
# 思路一 往左边添加，或者往右边添加
# 思路二 可以分解成更小的两个区间
# 思路三 看开始和结束，不要有影响
def interval_dp(self, nums: List[int]) :
    n = len(nums)
    dp = [[0] * n for _ in range(n+1)]
    for l in range(n): # 长度从小到大
        for i in range(n-l): # 以 i 为 开头
            j = i + l           # 以 j 为 终点
            for k in range(i,j): # 以 k 为分割点，进行分治
                # Todo 业务逻辑
                pass



In [ ]:
# 区间dp模板
def int_dp(s: str) -> int:
    n = len(s)
    dp = [[0] * n for _ in range(n)]
    for i in range(n - 1, -1, -1):
        dp[i][i] = 1
        for j in range(i + 1, n):
            if s[i] == s[j]:
                dp[i][j] = dp[i + 1][j - 1] + 2
            else:
                dp[i][j] = max(dp[i + 1][j], dp[i][j - 1])
    return dp[0][-1]




In [ ]:
# 0-1背包
## 二维数组
def knapsack(weights, values, capacity):
    n = len(weights)
    # dp[i][j] 表示前 i 个物品，背包容量为 j 时的最大价值
    dp = [[0] * (capacity + 1) for _ in range(n + 1)]

    for i in range(1, n + 1):
        for j in range(capacity + 1):
            if j >= weights[i - 1]:  # 如果当前背包容量 j 足够放第 i 个物品
                dp[i][j] = max(dp[i - 1][j], dp[i - 1][j - weights[i - 1]] + values[i - 1])
            else:
                dp[i][j] = dp[i - 1][j]  # 不放第 i 个物品

    return dp[n][capacity]  # 返回最大价值


# 示例用法
weights = [2, 3, 4, 5]
values = [3, 4, 5, 6]
capacity = 5
result = knapsack(weights, values, capacity)
print(result)  # 输出最大价值


## 空间优化
def knapsack(weights, values, capacity):
    n = len(weights)
    dp = [0] * (capacity + 1)

    for i in range(n):
        for j in range(capacity, weights[i] - 1, -1):  # 从后向前更新
            dp[j] = max(dp[j], dp[j - weights[i]] + values[i])

    return dp[capacity]

# 示例用法
weights = [2, 3, 4, 5]
values = [3, 4, 5, 6]
capacity = 5
result = knapsack(weights, values, capacity)
print(result)  # 输出最大价值


In [3]:
# 数位dp
## i 从0往高位， 高位是否限制了
low = str(10)
high = str(222)
n = len(high)
# low 需要保持数位一致，补充前导零
low = low.zfill(len(high))

# 不考虑前导零
@cache
def digit_dp(i, limit_low, limit_high):
    if i == n:
        return 1

    # i 的枚举范围， 这里不能改
    lo = int(low[i]) if limit_low else 0
    hi = int(high[i]) if limit_high else 9

    res = 0
    # 如果对数位有约束，可以在这里做限制
    for d in range(lo, hi + 1):
        res += digit_dp(i+1, limit_low and d == lo, limit_high and d == hi)
    return res

digit_dp(0, True, True)


# 有前导零 is_num 前面是否填了0
@cache
def digit_dp(i, limit_low, limit_high, is_num):
    if i == n:
        return 1
    res = 0
    # 前面为0 这里也可以是0, 肯定有下界的约束, 有前导零肯定比上界短，所以limit_high为False
    if not is_num and low[i] == '0':
        res += digit_dp(i + 1, True, False, False)

    # i 的枚举范围， 这里不能改
    lo = int(low[i]) if limit_low else 0
    hi = int(high[i]) if limit_high else 9


    # 如果对数位有约束，可以在这里做限制
    for d in range(lo, hi + 1):
        res += digit_dp(i+1, limit_low and d == lo, limit_high and d == hi)
    return res

digit_dp(0, True, True, False)

213

In [ ]:
# 树形dp
## 头结点到任何节点是唯一的
## 依赖关系比较简单：父依赖子

# 套路
## 父树需要的答案需要子树提供什么信息
## 把子树信息的全集定义成递归信返回
class TreeNode:
    def __init__(self, value):
        self.value = value
        self.children = []

def dfs(node):
    # 如果是叶子节点，返回自定义的值
    if not node.children:
        return node.value  # 可以是其他与问题相关的值

    # 记录子树的状态
    max_value = 0

    for child in node.children:
        max_value = max(max_value, dfs(child))  # 递归计算子节点的值

    # 结合当前节点的值，返回结果
    return max_value + node.value  # 根据需要进行调整

# 示例用法
root = TreeNode(1)
child1 = TreeNode(2)
child2 = TreeNode(3)
child1.children.append(TreeNode(4))
child1.children.append(TreeNode(5))
child2.children.append(TreeNode(6))
root.children.append(child1)
root.children.append(child2)

result = dfs(root)
print(result)  # 输出树的某种特征（如最大路径和）


In [ ]:
# 划分型dp
# dp[i][j]前i 划分成 j个

In [ ]:
# 换根dp
class TreeNode:
    def __init__(self, value):
        self.value = value
        self.children = []

def dfs1(node, parent):
    # 第一次 DFS 计算每个节点的子树信息
    subtree_value = node.value  # 当前节点的初始值

    for child in node.children:
        if child != parent:  # 避免回到父节点
            subtree_value += dfs1(child, node)  # 递归计算子树的值

    # 存储子树的值
    node.subtree_value = subtree_value
    return subtree_value

def dfs2(node, parent, total_value):
    # 第二次 DFS 计算换根后的值
    node.value = total_value  # 更新当前节点的值

    for child in node.children:
        if child != parent:
            # 计算换根后，子节点的值
            new_value = total_value - child.subtree_value + (total_value - node.value)
            dfs2(child, node, new_value)  # 递归计算

# 示例用法
root = TreeNode(1)
child1 = TreeNode(2)
child2 = TreeNode(3)
child1.children.append(TreeNode(4))
child1.children.append(TreeNode(5))
child2.children.append(TreeNode(6))
root.children.append(child1)
root.children.append(child2)

# 第一次 DFS 计算每个节点的子树值
dfs1(root, None)

# 第二次 DFS 从根节点开始，换根计算
total_value = root.subtree_value  # 根节点的总值
dfs2(root, None, total_value)

# 输出结果
def print_tree(node):
    print(f"Node Value: {node.value}")
    for child in node.children:
        print_tree(child)

print_tree(root)

In [ ]:
# 状态压缩dp 一般32位 样本量在20之内
def state_compression_dp(n):
    # dp[mask] 表示状态为 mask 时的最优解
    dp = [0] * (1 << n)  # 2^n 个状态

    for mask in range(1 << n):
        # 遍历当前状态 mask
        for i in range(n):
            if mask & (1 << i) == 0:  # 如果 i 还没有被选择
                new_mask = mask | (1 << i)  # 选择 i，更新状态
                # 根据具体问题更新 dp[new_mask]
                dp[new_mask] = max(dp[new_mask], dp[mask] + value(i, mask))  # 示例更新

    return dp[(1 << n) - 1]  # 返回所有状态都被选择时的结果

def value(i, mask):
    # 根据当前选择的 i 和状态 mask 计算价值
    # 这里可以根据具体问题进行实现
    return 1  # 示例：每次选择的价值都为 1

# 示例用法
n = 3  # 例如有 3 个物品
result = state_compression_dp(n)
print(result)  # 输出最优解

In [ ]:
# 线段树
## 区间长度很长，但是实际上要用很短 --> 离散化
## 区间问题如果只查询最后一次，那么其实用差分数组就行了
## 区间一直更新，还有一直查询，那就是线段树

In [ ]:
class SegmentTree:
    def __init__(self, nums):
        self.n = len(nums)
        self.tree = [0] * (4 * self.n)  # 线段树
        self.build(nums, 1, 0, self.n - 1)  # 从1开始的节点索引

    def pushup(self, left_part, right_part):
        return  max(left_part, right_part)
        # return left_part + right_part

    def build(self, nums, node, l, r):
        if l == r:
            self.tree[node] = nums[l]  # 从0索引转换为1索引
        else:
            mid = (l + r) // 2
            self.build(nums, 2 * node, l, mid)
            self.build(nums, 2 * node + 1, mid + 1, r)
            self.tree[node] = self.pushup(self.tree[2 * node], self.tree[2 * node + 1])

    def update(self, idx, value):
        self._update(1, 0, self.n - 1, idx, value)

    def _update(self, node, l, r, idx, value):
        if l == r:  # 叶子节点
            self.tree[node] = value
        else:
            mid = (l + r) // 2
            if l <= idx <= mid:
                self._update(2 * node, l, mid, idx, value)
            else:
                self._update(2 * node + 1, mid + 1, r, idx, value)
            self.tree[node] = self.pushup(self.tree[2 * node], self.tree[2 * node + 1])

    def query_range(self, L, R):
        return self._query_range(1, 0, self.n - 1, L, R)

    def _query_range(self, node, l, r, L, R):
        if R < l or L > r:  # 范围不交
            return 0
        if L <= l and R >= r:  # 完全包含
            return self.tree[node]

        mid = (l + r) // 2
        left_sum = self._query_range(2 * node, l, mid, L, R)
        right_sum = self._query_range(2 * node + 1, mid + 1, r, L, R)
        return self.pushup(left_sum, right_sum)

# 示例用法
nums = [1, 2, 3, 4, 5]
seg_tree = SegmentTree(nums)

# 更新索引 2 的值为 10 (0-indexed)
seg_tree.update(2, 10)

# 查询区间 [1, 4] (0-indexed)
result = seg_tree.query_range(1, 4)
print(result)  # 输出更新后的区间和

In [ ]:
# 懒更新线段树 区间更新
class LazySegmentTree:
    def __init__(self, nums):
        self.n = len(nums)
        self.tree = [0] * (4 * self.n)  # 线段树
        self.lazy = [0] * (4 * self.n)   # 懒标记
        self.build(nums, 1, 0, self.n - 1)  # 从1开始的节点索引

    def build(self, nums, node, l, r):
        if l == r:
            self.tree[node] = nums[l]  # nums 从0开始
        else:
            mid = (l + r) // 2
            self.build(nums, 2 * node, l, mid)
            self.build(nums, 2 * node + 1, mid + 1, r)
            self.tree[node] = self.tree[2 * node] + self.tree[2 * node + 1]

    def update_range(self, L, R, val):
        self._update_range(1, 0, self.n - 1, L, R, val)

    def _update_range(self, node, l, r, L, R, val):
        if self.lazy[node] != 0:  # 如果有懒更新
            self.tree[node] += (r - l + 1) * self.lazy[node]
            if l != r:
                self.lazy[2 * node] += self.lazy[node]
                self.lazy[2 * node + 1] += self.lazy[node]
            self.lazy[node] = 0

        if l > r or l > R or r < L:
            return

        if l >= L and r <= R:
            self.tree[node] += (r - l + 1) * val
            if l != r:
                self.lazy[2 * node] += val
                self.lazy[2 * node + 1] += val
            return

        mid = (l + r) // 2
        self._update_range(2 * node, l, mid, L, R, val)
        self._update_range(2 * node + 1, mid + 1, r, L, R, val)
        self.tree[node] = self.tree[2 * node] + self.tree[2 * node + 1]

    def query_range(self, L, R):
        return self._query_range(1, 0, self.n - 1, L, R)

    def _query_range(self, node, l, r, L, R):
        if l > r or l > R or r < L:
            return 0

        if self.lazy[node] != 0:  # 如果有懒更新
            self.tree[node] += (r - l + 1) * self.lazy[node]
            if l != r:
                self.lazy[2 * node] += self.lazy[node]
                self.lazy[2 * node + 1] += self.lazy[node]
            self.lazy[node] = 0

        if l >= L and r <= R:
            return self.tree[node]

        mid = (l + r) // 2
        left_sum = self._query_range(2 * node, l, mid, L, R)
        right_sum = self._query_range(2 * node + 1, mid + 1, r, L, R)
        return left_sum + right_sum

# 示例用法
nums = [1, 2, 3, 4, 5]
seg_tree = LazySegmentTree(nums)

# 更新区间 [1, 3] 加 10 (0-indexed)
seg_tree.update_range(1, 3, 10)

# 查询区间 [0, 4] (0-indexed)
result = seg_tree.query_range(0, 4)
print(result)  # 输出更新后的区间和

In [ ]:
# 动态开点线段树 单点更新，区间查询（求和）
class Node:
    def __init__(self):
        self.sum = 0        # 该区间的和
        self.left = None    # 左子节点
        self.right = None   # 右子节点

class DynamicSegmentTree:
    def __init__(self, nums):
        self.root = Node()
        self.nums = nums
        self.build(self.root, 0, len(nums) - 1)

    def build(self, node, l, r):
        if l == r:
            node.sum = self.nums[l]  # 叶子节点
        else:
            mid = (l + r) // 2
            node.left = Node()
            node.right = Node()
            self.build(node.left, l, mid)
            self.build(node.right, mid + 1, r)
            node.sum = node.left.sum + node.right.sum

    def update(self, idx, value):
        self._update(self.root, 0, len(self.nums) - 1, idx, value)

    def _update(self, node, l, r, idx, value):
        if l == r:  # 叶子节点
            node.sum = value
        else:
            mid = (l + r) // 2
            if idx <= mid:
                if node.left is None:
                    node.left = Node()
                self._update(node.left, l, mid, idx, value)
            else:
                if node.right is None:
                    node.right = Node()
                self._update(node.right, mid + 1, r, idx, value)
            node.sum = (node.left.sum if node.left else 0) + (node.right.sum if node.right else 0)

    def query_range(self, L, R):
        return self._query_range(self.root, 0, len(self.nums) - 1, L, R)

    def _query_range(self, node, l, r, L, R):
        if node is None or L > r or R < l:  # 范围不交
            return 0
        if L <= l and R >= r:  # 完全包含
            return node.sum

        mid = (l + r) // 2
        left_sum = self._query_range(node.left, l, mid, L, R)
        right_sum = self._query_range(node.right, mid + 1, r, L, R)
        return left_sum + right_sum

# 示例用法
nums = [1, 2, 3, 4, 5]
seg_tree = DynamicSegmentTree(nums)

# 更新索引 2 的值为 10 (0-indexed)
seg_tree.update(2, 10)

# 查询区间 [1, 4] (0-indexed)
result = seg_tree.query_range(1, 4)
print(result)  # 输出更新后的区间和

In [ ]:
# 字典树、前缀树模板
class TrieNode:
    def __init__(self):
        self.children = {}  # 存储子节点
        self.is_end_of_word = False  # 标记是否为单词的结尾

class Trie:
    def __init__(self):
        self.root = TrieNode()  # 初始化根节点

    def insert(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()  # 创建子节点
            node = node.children[char]
        node.is_end_of_word = True  # 标记单词的结束

    def search(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                return False  # 如果字符不存在，则返回 False
            node = node.children[char]
        return node.is_end_of_word  # 返回是否为单词的结尾

    def starts_with(self, prefix):
        node = self.root
        for char in prefix:
            if char not in node.children:
                return False  # 如果前缀不存在，则返回 False
            node = node.children[char]
        return True  # 如果所有字符都存在，则返回 True

# 示例用法
trie = Trie()
trie.insert("hello")
trie.insert("helicopter")

print(trie.search("hello"))      # 输出: True
print(trie.search("hell"))       # 输出: False
print(trie.starts_with("hel"))   # 输出: True

# 数论
# 除法同余
(a / b ) % mod = c
1. 保证 a / b 可以整除
2. 保证 mod 是质数
3. 保证 b 和 mod 互质（最大公约数为1）

逆元c =  (b ^(mod - 2)) % mod
(a * c) % d

# 费马小定理


# 裴蜀定理
ax %b = 1   (a ， b 互质)
ax -b * k = 1
x = (x % b + b) % b

ax + by = gcd(a, b) 对任意x，y 存在有gcd
最小正整数 gcd(a, b)

# 拓展欧几里得算法求逆元
ax + by = gcd(a, b)



In [ ]:
# 拓展欧几里得算法，已知a, b 求 x
def extended_gcd(a, b):
    if b == 0:
        return a, 1, 0  # 返回 gcd, x, y
    gcd, x1, y1 = extended_gcd(b, a % b)  # 递归调用
    x = y1  # 更新 x
    y = x1 - (a // b) * y1  # 更新 y
    return gcd, x, y

In [ ]:
# kmp 算法
def build_lps(pattern):
    lps = [0] * len(pattern)
    j = 0  # 前缀匹配的长度

    for i in range(1, len(pattern)):
        while j > 0 and pattern[i] != pattern[j]:
            j = lps[j-1]
        if pattern[i] == pattern[j]:
            j += 1
        lps[i] = j

    return lps

def kmp(text, pattern):
    if not pattern:
        return 0

    lps = build_lps(pattern)
    j = 0  # 模式串匹配的长度

    for i in range(len(text)):
        while j > 0 and text[i] != pattern[j]:
            j = lps[j-1]
        if text[i] == pattern[j]:
            j += 1
        if j == len(pattern):
            return i - j + 1

    return -1

# 测试代码
text = "ABABDABACDABABCABAB"
pattern = "ABABCABAB"
result = kmp(text, pattern)
print(result)  # 输出10，模式串在文本串中的起始位置

In [ ]:
# z函数
def calculate_z_function(s):
    n = len(s)
    z = [0] * n
    l = r = 0

    for i in range(1, n):
        if i <= r:
            z[i] = min(r - i + 1, z[i - l])
        while i + z[i] < n and s[z[i]] == s[i + z[i]]:
            z[i] += 1
        if i + z[i] - 1 > r:
            l, r = i, i + z[i] - 1

    return z

# 测试代码
string = "abacaba"
z_values = calculate_z_function(string)
print(z_values)  # 输出 [0, 0, 1, 0, 3, 0, 1]